# Automatic Differentiation
Automatic Differentiation (AD) is a powerful computational tool that has revolutionized various fields, particularly in mathematics, physics, engineering, and machine learning. It provides a systematic and efficient way to compute derivatives of complex mathematical functions, allowing us to effortlessly obtain accurate and reliable gradients. This capability of AD brings a multitude of benefits, enabling faster and more robust optimization, enhancing the performance of machine learning models, and facilitating the solution of intricate differential equations.

In this exercise you have to use both manual differentiation and automatic differentiation to find the solution of the following Poisson's equation.

$$-div(q(u)*\Delta(u)) = 0,$$
$$u = 0\text{ at }x=0, u=1\text{ at }x=1$$
$$q(u) = (1+2u+4u^3)$$

In [1]:
from dolfin import *
import numpy

mesh = IntervalMesh(40, 0, 1)
V = FunctionSpace(mesh, 'Lagrange', 1)

left_boundary = CompiledSubDomain("on_boundary && near(x[0],0)")
right_boundary = CompiledSubDomain("on_boundary && near(x[0],1)")

bc_0 = DirichletBC(V, Constant(0.0), left_boundary)
bc_1 = DirichletBC(V, Constant(1.0), right_boundary)
bcs = [bc_0, bc_1]

m = 2


def q(u):
    return 1 + 2*u + 4*(u**3)

def Dq(u):
    return 2 + 12*(u**2)

# Define variational problem
v = TestFunction(V)
du = TrialFunction(V)
u = Function(V)  # most recently computed solution
F = inner(q(u)*nabla_grad(u), nabla_grad(v))*dx

J_m = inner(q(u)*nabla_grad(du), nabla_grad(v))*dx + \
    inner(Dq(u)*du*nabla_grad(u), nabla_grad(v))*dx                 

J_a = derivative(F, u, du)    # auto_differentiation

# Compute solution1
problem1 = NonlinearVariationalProblem(F, u, bcs, J_m)
solver1 = NonlinearVariationalSolver(problem1)

prm = solver1.parameters
prm['newton_solver']['absolute_tolerance'] = 1E-5
prm['newton_solver']['relative_tolerance'] = 1E-5
prm['newton_solver']['maximum_iterations'] = 25

solver1.solve()

(4, True)

In [2]:
u_m = Function(V)
u_m.assign(u)

In [3]:
# Compute solution2
problem2 = NonlinearVariationalProblem(F, u, bcs, J_a)
solver2 = NonlinearVariationalSolver(problem2)

prm = solver2.parameters
prm['newton_solver']['absolute_tolerance'] = 1E-5
prm['newton_solver']['relative_tolerance'] = 1E-5
prm['newton_solver']['maximum_iterations'] = 25

solver2.solve()

Calling FFC just-in-time (JIT) compiler, this may take some time.


(1, True)

In [4]:
u_a = Function(V)
u_a.assign(u)

In [5]:
diff = numpy.abs(u_m.vector()[:] - u_a.vector()[:]).max()
print('Max error:{0:5.3e}'.format(diff))

Max error:1.743e-05
